In [5]:
from fredapi import Fred
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller,grangercausalitytests
from pandas_datareader.data import DataReader
from statsmodels.tsa.seasonal import seasonal_decompose
from datetime import date, datetime
from openpyxl import load_workbook

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pandas as pd
import numpy as np

import plotly.graph_objects as go
import seaborn as sns



fred = Fred(api_key='7e70df3f020852ea824275ce0fea70a0')

## Quarterly data to replicate the working paper

In [35]:
def parser(x):
    return datetime.strptime('190'+x, '%Y-%m')

# monthly CPI


series_list = [
         'GDPC1', # Real GDP, Q1 1947 to Q3 2022
         'A794RX0Q048SBEA', # Real personal consumption expenditures per capita, Q1 1947 to Q3 2022
    
    
    # Real private fixed investment (chain-type price index) 
          'B011RG3Q086SBEA', # Residential, Q1 1947 to Q3 2022
          'B008RG3Q086SBEA', # Non residential, Q1 1947 to Q3 2022
    
    # Real Trade of Goods and Services
          'EXPGSC1', # Exports, Q1 1947 to Q3 2022
          'IMPGSC1', # Imports, Q1 1947 to Q3 2022
    
   
    
    # Real gross private domestic investment: Fixed investment (chain-type quantity index)
          'A011RA3Q086SBEA', #  Q1 1947 to Q3 2022
          'B824RA3Q086SBEA', # Federal: National defense, Q1 1947 to Q3 2022
    
    # Chain type price index
          'GDPCTPI', # Gross Domestic Product, Q1 1947 to Q3 2022
          'PCEPI', # PCE, Jan 1959 to Sep 2022
          'JCXFE', # PCE excluding food and energy, Q1 1959 to Q3 2022
    
    # CPI for all urban consumers
          'CPIAUCSL', # all items Jan 1947 to Oct 2022
          'CPILFESL', # all items except food and energy, Jan 1957 to Oct 2022
    
    # Compensation of employees
          'A576RC1', # Received: Wage and Salary Disbursements, Jan 1959 to Sep 2022
          'A442RC1Q027SBEA', # Net value added of corporate business , Q1 1947 to Q3 2022
    
          'UNRATE', # Unemployment rate, Jan 1948 to Oct 2022
          'INDPRO', # Industrial Production Index, Jan 1919 to Oct 2022 
          'CUMFNS', # Capacity Utilization: Manufacturing (SIC), Jan 1948 to Oct 2022
          'HOUST', # New Privately-Owned Housing Units Started: Total Units, Jan 1959 to Oct 2022 
          'DSPIC96', # Real Disposable Personal Income, Jan 1959 to Sep 2022
          'UMCSENT', # University of Michigan: Consumer Sentiment, Nov 1952 to Oct 2022
    
    # Treasury Notes
          'DGS1', # 1 year, 1962-01-02 to 2022-11-23 
          'DGS5', # 5 year, 1962-01-02 to 2022-11-23
          'DGS10', # 10 year, 1962-01-02 to 2022-11-23
    
    # Moody's Seasoned Corporate Bond Yield
          'DAAA', # Aaa, 1983-01-03 to 2022-11-23
          'DBAA', # Baa, 1986-01-02 to 2022-11-23
    
          'USASARTMISMEI', # Total Retail Trade in US, Jan 1960 to Aug 2022
          'WILL5000PRFC', # Wilshire 5000 Full Cap Price Index, 1970-12-31 to 2022-11-23
          'WTISPLC', # Spot Crude Oil Price: West Texas Intermediate (WTI), Jan 1946 to Oct 2022
          'PPIACO', # PPI, Jan 1913 to Oct 2022
          'EMVCOMMMKT', # Equity Market Volatility Tracker: Commodity Markets, Jan 1985 to Oct 2022
    
    # Equity Market Volatility Tracker
    
          'WLEMUINDXD' # Equity Market-related Economic Uncertainty Index,  1985-01-01 to 2023-03-28
 
    
         ] 


In [36]:
 # series has quarterly frequency variables
# save output in a sheet of an excel files



# Define start and end dates
start_date = '1986-01-01'
end_date = '2022-08-01'
excel_file_name = 'data_SJ.xlsx'
sheet_name = 'Data'


 # Retrieve the data for each series ID
data = {}
for series_id in series_list:
    data[series_id] = fred.get_series(series_id,observation_start= start_date, observation_end=end_date ,
                 frequency ='q')


# create a new sheet in the Excel file using the pd.ExcelWriter() method, 
# with the vintage date as the sheet name. 

with pd.ExcelWriter(excel_file_name, mode='a', engine='openpyxl') as writer:
    writer.book = load_workbook(excel_file_name)


    # If a sheet with the same name already exists in the Excel file,
   # remove it first before creating the new sheet. 

    if sheet_name in writer.book.sheetnames:
        writer.book.remove(writer.book[sheet_name])
    df = pd.DataFrame(data)
    df.insert(0, 'Date', df.index)

    # save the data to the new sheet 
    df.to_excel(writer, sheet_name=sheet_name, index=False)

## Monthly data for my own analysis


In [2]:
def parser(x):
    return datetime.strptime('190'+x, '%Y-%m')

# monthly data

series_list = [
    
      
      'INDPRO', # Industrial Production Index, Jan 1919 to Nov 2022 
      'CUMFNS', # Capacity Utilization: Manufacturing (SIC), Jan 1948 to Nov 2022
      'HOUST', # New Privately-Owned Housing Units Started: Total Units, Jan 1959 to Nov 2022 
   #   'CFNAIDIFF', # Chicago Fed National Activity Index: Diffusion Index, May 1967 to Feb 2023 
    
      'DSPIC96', # Real Disposable Personal Income, Jan 1959 to Nov 2022
      'PDI', # Personal Income Receipts on Assets: Personal Dividend Income, Jan 1959 to Nov 2022, (billions of USD)
     'BAA10Y', # Moody's Seasoned Baa Corporate Bond Yield Relative to Yield on 10-Year Treasury Constant Maturity
      'UMCSENT', # University of Michigan: Consumer Sentiment, Nov 1952 to Nov 2022
    
    
      'UNRATE', # Unemployment rate, Jan 1948 to Nov 2022
      'WPU058103', # PPI by Commodity: Fuels and Related Products and Power, Jul 1985 to Nov 2022
    
     
      'A576RC1', # Compensation of employees Received: Wage and Salary Disbursements, Jan 1959 to Nov 2022
      'EMRATIO', # Employment-Population Ratio, Jan 1948 to Nov 2022
    
    
      'WPSFD4111', # PPI by Commodity, Final demand: finished consumer goods, Apr 1947 to Nov 2022
    
    # Chain type price index
      'PCEPI', # PCE, Jan 1959 to Nov 2022
     'PCEPILFE', # PCE excluding food and energy, Jan 1959 to Feb 2023 
    
      'PPIACO', # PPI: All Commodities, Jan 1913 to Nov 2022
      
    
    # CPI for all urban consumers
      'CPIAUCSL', # all items Jan 1947 to Nov 2022
      'CPILFESL', # all items except food and energy, Jan 1957 to Nov 2022
     
 
    # Rates
      'DFF', # federal funds rate, 1954-07-01 to 2022-12-29
      'T10Y2Y', # 10-Year Minus 2-Year Treasury Constant Maturity, 1976-06-01 to 2023-01-03
    
    # Treasury Notes 
      'DGS1', # 1 year, 1962-01-02 to 2022-12-29
      'DGS5', # 5 year, 1962-01-02 to 2022-12-29
      'DGS10', # 10 year, 1962-01-02 to 2022-12-29
    
    # Moody's Seasoned Corporate Bond Yield
      'DAAA', # Aaa, 1983-01-03 to 2022-12-29
      'DBAA', # Baa, 1986-01-02 to 2022-12-29
    
    
      'NASDAQCOM', # NASDAQ Composite Index, 1971-02-05 to 2022-12-29
      'WTISPLC', # Spot Crude Oil Price: West Texas Intermediate (WTI), Jan 1946 to Nov 2022
      
    
    # spot exchange rate: x currency to USD at the spot market (daily)
    
     'DEXUSUK', # U.K. Pound Sterling Spot Exchange Rate, 1971-01-04 to 2022-12-23
     'DEXJPUS', # Japanese yen, 1971-01-04 to 2022-12-23
     'DEXCAUS', # Canadian dollar, 1971-01-04 to 2022-12-23
     'RNUSBIS', # Real Narrow Effective Exchange Rate, Jan 1964 to Nov 2022
    
    # Equity Market Volatility Tracker
    
    'EMVCOMMMKT', # Commodity Markets, Jan 1985 to Nov 2022
    'WLEMUINDXD', # Equity Market-related Economic Uncertainty Index,  1985-01-01 to 2023-03-28
    'EMVMACROINTEREST', # Macroeconomic News and Outlook: Interest Rates,
                             # Jan 1985 to Feb 2023
    'EMVMONETARYPOL', # Monetary Policy, Jan 1985 to Feb 2023 
    'EMVGOVTSPEND', # Government Spending Deficits And Debt, Jan 1985 to Feb 2023
    'EMVFINCRISES', # Financial Crises, Jan 1985 to Feb 2023 
    'INFECTDISEMVTRACKD', # Equity Market Volatility: Infectious Disease Tracker, 1985-01-01 to 2023-04-02 

    # Rates and Spread
    'DPRIME',# Bank Prime Loan Rate, 1955-08-04 to 2023-03-28 
    'T6MFF', # 6-Month Treasury Constant Maturity Minus Federal Funds Rate, 1982-01-04 to 2023-03-28 
    
    # Treasuries
    
    'TREASURY', # Treasury Deposits with Federal Reserve Banks, Jan 1986 to Mar 2023
    'TASACBW027SBOG', # Treasury and Agency Securities, All Commercial Banks, 1973-01-03 to 2023-03-22
    
    'MVMTD027MNFRBDAL',# Market Value of Marketable Treasury Debt, Jan 1942 to Feb 2023
    'HQMCB10YR' # 10-Year High Quality Market (HQM) Corporate Bond Spot Rate, Jan 1984 to Feb 2023    
    
 
         ] 



In [3]:
# series has monthly and daily frequency variables
# save output in one excel files



# Define start and end dates
start_date = '1986-01-01'
end_date = '2023-05-01'
excel_file_name = 'Data Monthly.xlsx'
sheet_name = 'Big Data'



 # Retrieve the data for each series ID
data = {}
for series_id in series_list:
    data[series_id] = fred.get_series(series_id,observation_start= start_date, observation_end=end_date ,
                 frequency ='m', aggregation_method = 'avg')


# create a new sheet in the Excel file using the pd.ExcelWriter() method, 
# with the vintage date as the sheet name. 

with pd.ExcelWriter(excel_file_name, mode='a', engine='openpyxl') as writer:
    writer.book = load_workbook(excel_file_name)


    # If a sheet with the same name already exists in the Excel file,
   # remove it first before creating the new sheet. 

    if sheet_name in writer.book.sheetnames:
        writer.book.remove(writer.book[sheet_name])
    df = pd.DataFrame(data)
    df.insert(0, 'Date', df.index)

    # save the data to the new sheet 
    df.to_excel(writer, sheet_name=sheet_name, index=False)
    

## Financial Conditions Index

In [44]:
series_list = [
     'KCFSI', # Kansas City Financial Stress Index, monthly, Feb 1990 to Feb 2023
     'STLFSI4', # St. Louis Fed Financial Stress Index, weekly, 1993-12-31 to 2023-03-24
     'NFCI' # Chicago Fed National Financial Conditions Index, weeky, 1971-01-08 to 2023-03-24 
 
         ] 


# series has monthly and daily frequency variables
# save output in one excel files

# Define start and end dates
start_date = '1994-01-01'
end_date = '2023-03-01'

# Define the excel file and spread sheet name within the excel file
excel_file_name = 'Data Monthly.xlsx'
sheet_name = 'FCI'



 # Retrieve the data for each series ID
data = {}
for series_id in series_list:
    data[series_id] = fred.get_series(series_id,observation_start= start_date, observation_end=end_date ,
                 frequency ='m', aggregation_method = 'avg')


# create a new sheet in the Excel file using the pd.ExcelWriter() method, 
# with the vintage date as the sheet name. 

with pd.ExcelWriter(excel_file_name, mode='a', engine='openpyxl') as writer:
    writer.book = load_workbook(excel_file_name)


    # If a sheet with the same name already exists in the Excel file,
   # remove it first before creating the new sheet. 

    if sheet_name in writer.book.sheetnames:
        writer.book.remove(writer.book[sheet_name])
    df = pd.DataFrame(data)
    df.insert(0, 'Date', df.index)

    # save the data to the new sheet 
    df.to_excel(writer, sheet_name=sheet_name, index=False)

## Variables on international trade and exchanges rates to create an exchange rate index¶

In [15]:
# Create a list of series IDs to retrieve

series_list = [
    
    ######################### Real Activity and Manufacturing #########################
    
   
      'INDPRO', # Industrial Production Index, Jan 1919 to Nov 2022 
      'CUMFNS', # Capacity Utilization: Manufacturing (SIC), Jan 1948 to Nov 2022
 
    
   
    
    ################### Housing and Construction #######################################
    
      'HOUST', # New Privately-Owned Housing Units Started: Total Units, Jan 1959 to Nov 2022 
      'HSN1F', # New One Family Houses Sold, Thousands, Monthly, Jan 1963 to Mar 2023   
   
    
    ####################################### Labor #######################################
    
   
      'UNRATE', # Unemployment rate, Jan 1948 to Nov 2022
      'A576RC1', # Compensation of employees Received: Wage and Salary Disbursements, Jan 1959 to Nov 2022
      'EMRATIO', # Employment-Population Ratio, Jan 1948 to Nov 2022
  
    
     ####################################### Prices #######################################
    
   
   
    
    # CPI for all urban consumers
      'CPIAUCSL', # all items Jan 1947 to Nov 2022
      'CPILFESL', # all items except food and energy, Jan 1957 to Nov 2022
     
      'DFF', # federal funds rate, 1954-07-01 to 2022-12-29
    
   
    
     ###################### International Trade #######################################
    
    # Trade Balance: Goods and Services, Balance of Payments Basis, 
       'BOPGSTB', # Millions of Dollars, Monthly, Jan 1992 to Mar 2023
    
    # Exports of Goods and Services, Balance of Payments Basis, Millions of Dollars, 
               # Monthly, Jan 1992 to Feb 2023        
       'BOPTEXP', 
               
       #  Exports of Goods and Services, Balance of Payments Basis, Millions of Dollars, 
               # Monthly, Jan 1992 to Feb 2023           
        'BOPTIMP',
 
       # Import Price Index (End Use): All Commodities, Index 2000=100, Monthly, Sep 1982 to Mar 2023        
         'IR', 
               
       # Export Price Index (End Use): All Commodities, Index 2000=100, Monthly, Sep 1983 to Mar 2023        
         'IQ',
    

    
    # spot exchange rate: x currency to USD at the spot market (daily)
    
     'DEXUSUK', # USD to U.K. Pound Sterling Spot Exchange Rate, 1971-01-04 to 2022-12-23
     'DEXJPUS', # Japanese yen, 1971-01-04 to 2022-12-23
     'DEXCAUS', # Canadian dollar, 1971-01-04 to 2022-12-23
     'DEXCHUS', # Chinese yuan renminbi, 1981-01-02 to 2023-05-05
   
     'DEXINUS', # Indian rupees, 1973-01-02 to 2023-05-05
     'DEXSZUS', # Swiss francs, 1971-01-04 to 2023-05-05
    
     'DEXSIUS', # Singapore Dollars, 1981-01-02 to 2023-05-05
     'DEXUSAL', # USD to Australian dollar, 1971-01-04 to 2023-05-05
     'DEXMAUS', # Malaysian Ringgit, 1971-01-04 to 2023-05-05
    
     'DEXTHUS', # Thai Baht, 1981-01-02 to 2023-05-05
     'CCUSSP02IDM650N', # Indonesian rupiah, Jan 1967 to Mar 2023
     'DEXSDUS', # Swedish Kronor, 1971-01-04 to 2023-05-05
     'DEXKOUS', # South Korean Won, 1981-04-13 to 2023-05-05
     'DEXTAUS', # Taiwan Dollars, 1983-10-03 to 2023-05-05 
     'DEXUSEU', # Euro, 1999-01-04 to 2023-05-05
    
    
    
     #### Newely added spot exchange rates
     'DEXHKUS', # Hong Kong Dollars, 1981-01-02 to 2023-05-19
     'DEXBZUS', # Brazilian Reals
     'CCUSSP02ILM650N', # New Israeli Sheqel, 
    
    
    
    # US Exports of goods by FAS basis to country x: Millions of Dollars, Monthly, Jan 1985 to Mar 2023 
    'EXPCH', # China, 
    'EXPUK', # U.K,  Jan 1960 to Feb 2023
    'EXPJP', # Japan, 
    'EXP5330', # India
    'EXP4419', # Switzerland
    'EXP5590', # Singapore
    'EXP6021', # Australia
    'EXP5570', # Malaysia
    'EXP4010', # Sweden
    'EXP0003', # EU
    'EXPKR', # South Korea
    'EXP5830', # Taiwan
    
    
    ### Newely added exports
     'EXP5820', # Hong Kong
     'EXP3510', # Brazil
     'EXP5081', # Israel
    
    
    # U.S. Imports of Goods by Customs Basis from country x
    # Millions of Dollars, Monthly,Jan 1985 to Mar 2023
    'IMPJP', #  Japan 
    'IMPCA', # Canada 
    'IMPCH', # China
    'IMP5330', # India
    'IMP4419', # Switzerland
    'IMPUK',   # UK
    'IMP5590', # Singapore
    'IMP6021', # Australia
    'IMP5570', # Malaysia
    'IMP5600', # Indonesia
    'IMP4010', # Sweden
    'IMP0003', # European Union
    'IMPKR',   # South Korea
    'IMP5830' , # Taiwan
   
    
     ### Newely added imports
      'IMP5820', # Hong Kong
      'IMP3510', # Brazil
      'IMP5081' # Israel
   
 
         ] 

In [16]:
# series has monthly and daily frequency variables
# save output in one excel files

# Define start and end dates
start_date = '1999-01-01'
end_date = '2023-05-01'

# Define the excel file and spread sheet name within the excel file
excel_file_name = 'Data Monthly.xlsx'
sheet_name = 'FX and Trade'



 # Retrieve the data for each series ID
data = {}
for series_id in series_list:
    data[series_id] = fred.get_series(series_id,observation_start= start_date, observation_end=end_date ,
                 frequency ='m', aggregation_method = 'avg')


# create a new sheet in the Excel file using the pd.ExcelWriter() method, 
# with the vintage date as the sheet name. 

with pd.ExcelWriter(excel_file_name, mode='a', engine='openpyxl') as writer:
    writer.book = load_workbook(excel_file_name)


    # If a sheet with the same name already exists in the Excel file,
   # remove it first before creating the new sheet. 

    if sheet_name in writer.book.sheetnames:
        writer.book.remove(writer.book[sheet_name])
    df = pd.DataFrame(data)
    df.insert(0, 'Date', df.index)

    # save the data to the new sheet 
    df.to_excel(writer, sheet_name=sheet_name, index=False)

### REER and Trade Weighted Dollar Index from FRED to Compare with BVAR created FX Index

In [17]:
# Create a list of series IDs to retrieve

series_list = [
    
    'RTWEXBGS', # Real Broad Dollar Index, Index Jan 2006=100, monthly, Jan 2006 to Apr 2023
    'RBUSBIS', # Real Broad Effective Exchange Rate for United States, monthly, Jan 1994 to Mar 2023
    'TWEXM', # Trade Weighted U.S. Dollar Index: Major Currencies, Goods, weekly, 1973-01-03 to 2020-01-01
    'DTWEXBGS' # Nominal Broad U.S. Dollar Index, Index Jan 2006=100, Daily, 2006-01-02 to 2023-05-12
    
]

# series has monthly and weekly frequency variables
# save output in one excel files

# Define start and end dates
start_date = '1994-01-01'
end_date = '2023-05-01'

# Define the excel file and spread sheet name within the excel file
excel_file_name = 'Data Monthly.xlsx'
sheet_name = 'Dollar Index'



 # Retrieve the data for each series ID
data = {}
for series_id in series_list:
    data[series_id] = fred.get_series(series_id,observation_start= start_date, observation_end=end_date ,
                 frequency ='m', aggregation_method = 'avg')


# create a new sheet in the Excel file using the pd.ExcelWriter() method, 
# with the vintage date as the sheet name. 

with pd.ExcelWriter(excel_file_name, mode='a', engine='openpyxl') as writer:
    writer.book = load_workbook(excel_file_name)


    # If a sheet with the same name already exists in the Excel file,
   # remove it first before creating the new sheet. 

    if sheet_name in writer.book.sheetnames:
        writer.book.remove(writer.book[sheet_name])
    df = pd.DataFrame(data)
    df.insert(0, 'Date', df.index)

    # save the data to the new sheet 
    df.to_excel(writer, sheet_name=sheet_name, index=False)


## Financial Intermediation

In [6]:
def parser(x):
    return datetime.strptime('190'+x, '%Y-%m')

# data from FRED

series_list = [
    
      
      'INDPRO', # Industrial Production Index, Jan 1919 to Nov 2022 
      'CPILFESL', # CPI: all items except food and energy, Jan 1957 to Nov 2022', 
      'UNRATE', # Unemployment rate, Jan 1948 to Nov 2022
      'PPIACO', # PPI: All Commodities, Jan 1913 to Nov 2022
      'UMCSENT', # University of Michigan: Consumer Sentiment, Nov 1952 to Nov 2022
      'WTISPLC', # Spot Crude Oil Price: West Texas Intermediate (WTI), Jan 1946 to Nov 2022
     'NASDAQCOM', # NASDAQ Composite Index, 1971-02-05 to 2022-12-29
    
    
    
        # spot exchange rate: x currency to USD at the spot market (daily)
    
     'DEXUSUK', # U.K. Pound Sterling Spot Exchange Rate, 1971-01-04 to 2022-12-23
     'DEXJPUS', # Japanese yen, 1971-01-04 to 2022-12-23
     'DEXCAUS', # Canadian dollar, 1971-01-04 to 2022-12-23
     'RNUSBIS', # Real Narrow Effective Exchange Rate, Jan 1964 to Nov 2022
    
    
    
    # Treasury Notes 
     
      'DGS2', # 2 year, 1962-01-02 to 2022-12-29
      'DGS5', # 5 year, 1962-01-02 to 2022-12-29
      'DGS10', # 10 year, 1962-01-02 to 2022-12-29
      'T1YFF', # 1 year Treasury bill minus the federal funds rate 
      'DFF',   # Federal funds rate
    
    # Money type
      
     'M1SL', # M1, Jan 1959 to Apr 2023
     'M2REAL', # Real M2 money stock, Jan 1959 to Apr 202 
     'MABMM301USM189S', # M3, monthly, Jan 1960 to Mar 2023
    
    
    # Nonfinancial Corporate Business (QUARTERLY)
    
    'BCNSDODNS', # Debt Securities and Loans; Liability, # Q4 1945 to Q1 2023
    'NCBEILQ027S', # Corporate Equities; Liability, Level, Q4 1945 to Q1 2023
    
    
    
    ############ Assets of Commercial Banks ##################
    
    
   # Loans by Commercial Banks, weekly
    
     'CLSACBW027SBOG', # Consumer Loans, 1973-01-03 to 2023-05-31
     'RELACBW027SBOG', # Real Estate Loans, 1973-01-03 to 2023-05-31
     'BUSLOANS', # Commercial and Industrial Loans, 1973-01-03 to 2023-06-0
    
    # Securities and liabilities (borrowings) of Commercial banks, weekly
    
     'CASACBW027SBOG', # Cash assets, 1973-01-03 to 2023-06-07 
     'SBCACBW027SBOG', # Securities in bank credit, 1973-01-03 to 2023-06-07
     'TASACBW027SBOG', # Treasury and Agency Securities, 1973-01-03 to 2023-06-07 
     'H8B3094NCBA', # Borrowings, 1973-01-03 to 2023-06-07
    
    
    
     'DPRIME',# Bank Prime Loan Rate, 1955-08-04 to 2023-03-28 
     'MORTGAGE30US', # 30-Year Fixed Rate Mortgage Average, 1971-04-02 to 2023-06-08, weekly 
    
    
    'LTDACBW027SBOG', # Large Time Deposits, All Commercial Banks
    'PMSAVE' # Personal Savings, Jan 1959 to Apr 2023
 
 
         ] 


In [7]:
# Retrieve the data for each series ID

# Define start and end dates
start_date = '1978-01-01'
end_date = '2023-03-01'

# Define the excel file and sheet name within the excel file
excel_file_name = 'Data Monthly.xlsx'
sheet_name = 'FinInter'


data = {}
for series_id in series_list:
    frequency = 'm'
    
    if series_id in ['BCNSDODNS','NCBEILQ027S'] :
        frequency = 'q'
    series_data = fred.get_series(series_id, observation_start=start_date,
       observation_end=end_date, frequency=frequency)

    if frequency == 'd':
        # Aggregate daily data to monthly by taking the average of all the daily data for each month
        series_data = series_data.resample('M').mean()
        
    if frequency == 'w':    
        series_data = series_data.resample('M').mean()
        
    elif frequency == 'q':
        # Store quarterly data in the first month of each quarter and let the remaining months 
        # have no or missing values
        series_data = series_data.asfreq('MS')
        # Interpolate the quarterly data to fill in missing monthly values
        series_data = series_data.interpolate('linear')
        
    data[series_id] = series_data
    

# create a new sheet in the Excel file using the pd.ExcelWriter() method, 
# with the vintage date as the sheet name. 

with pd.ExcelWriter(excel_file_name, mode='a', engine='openpyxl') as writer:
    writer.book = load_workbook(excel_file_name)

    # Remove the existing sheet if it exists
    if sheet_name in writer.book.sheetnames:
        writer.book.remove(writer.book[sheet_name])
    
    df = pd.DataFrame(data)
    df.insert(0, 'Date', df.index)

    # save the data to the new sheet 
    df.to_excel(writer, sheet_name=sheet_name, index=False)


## Stress Testing

In [12]:
def parser(x):
    return datetime.strptime('190'+x, '%Y-%m')

# monthly data

series_list = [
    
      
      'INDPRO', # Industrial Production Index, Jan 1919 to Nov 2022 
      'HOUST', # New Privately-Owned Housing Units Started: Total Units, Jan 1959 to Nov 2022 

      'DSPIC96', # Real Disposable Personal Income, Jan 1959 to Nov 2022
 

      'UNRATE', # Unemployment rate, Jan 1948 to Nov 2022

    
    # Chain type price index
      'PCEPI', # PCE, Jan 1959 to Nov 2022
     'PCEPILFE', # PCE excluding food and energy, Jan 1959 to Feb 2023 
    
    # CPI for all urban consumers
      'CPIAUCSL', # all items Jan 1947 to Nov 2022
      'CPILFESL', # all items except food and energy, Jan 1957 to Nov 2022
     
 
    # Rates
      'DFF', # federal funds rate, 1954-07-01 to 2022-12-29
      'T10Y2Y', # 10-Year Minus 2-Year Treasury Constant Maturity, 1976-06-01 to 2023-01-03
    
    # Treasury Notes 
      'DGS3MO', # 3 month, 1981-09-01 to 2023-06-14 
      'DGS5', # 5 year, 1962-01-02 to 2022-12-29
      'DGS10', # 10 year, 1962-01-02 to 2022-12-29
    
    # Moody's Seasoned Corporate Bond Yield
      'DAAA', # Aaa, 1983-01-03 to 2022-12-29
      'DBAA', # Baa, 1986-01-02 to 2022-12-29
    'BAA10Y', # Moody's Seasoned Baa Corporate Bond Yield Relative to 10 year Treasury, 1986-01-02 to 2023-06-14
    
    
      'NASDAQCOM', # NASDAQ Composite Index, 1971-02-05 to 2022-12-29
  
      
    
    # spot exchange rate: x currency to USD at the spot market (daily)
    
     'DEXUSUK', # U.K. Pound Sterling Spot Exchange Rate, 1971-01-04 to 2022-12-23
     'DEXJPUS', # Japanese yen, 1971-01-04 to 2022-12-23
     'DEXCAUS', # Canadian dollar, 1971-01-04 to 2022-12-23
     'RNUSBIS', # Real Narrow Effective Exchange Rate, Jan 1964 to Nov 2022

    'WLEMUINDXD', # Equity Market-related Economic Uncertainty Index,  1985-01-01 to 2023-03-28
  

    # Rates and Spread
    'DPRIME',# Bank Prime Loan Rate, 1955-08-04 to 2023-03-28 
    'MORTGAGE30US', # 30-Year Fixed Rate Mortgage Average, 1971-04-02 to 2023-06-08, weekly 
    'T6MFF', # 6-Month Treasury Constant Maturity Minus Federal Funds Rate, 1982-01-04 to 2023-03-28 
    
    
    ####### Foreign countries: CPI Inflation: All Items   #######
    
    'JPNCPIALLMINMEI', # Japan, Jan 1960 to Apr 2022
    'GBRCPIALLMINMEI', # United Kingdom, Jan 1960 to Mar 2023
    'CPALCY01CAM661N', # Canada, Jan 1961 to Mar 2023
    
    # Production of Total Industry
    
    'JPNPROINDMISMEI', #  Japan, Jan 1960 to Feb 2023
    'GBRPROINDMISMEI', # United Kingdom, Jan 1968 to Feb 2023
    
    'CANPROINDMISMEI', # Total Industry Excluding Construction for Canada, Jan 1961 to Feb 2023
    
    

     # Money type
      
     'M1SL', # M1, Jan 1959 to Apr 2023
     'M2REAL', # Real M2 money stock, Jan 1959 to Apr 202 
     'MABMM301USM189S', # M3, monthly, Jan 1960 to Mar 2023
    
     ############ Assets of Commercial Banks ##################
    
    
   # Loans by Commercial Banks, weekly
    
     'CLSACBW027SBOG', # Consumer Loans, 1973-01-03 to 2023-05-31
     'RELACBW027SBOG', # Real Estate Loans, 1973-01-03 to 2023-05-31
     'BUSLOANS', # Commercial and Industrial Loans, 1973-01-03 to 2023-06-0
    
    # Securities and liabilities (borrowings) of Commercial banks, weekly
    
     'CASACBW027SBOG', # Cash assets, 1973-01-03 to 2023-06-07 
     'SBCACBW027SBOG', # Securities in bank credit, 1973-01-03 to 2023-06-07
     'TASACBW027SBOG', # Treasury and Agency Securities, 1973-01-03 to 2023-06-07 
     'H8B3094NCBA', # Borrowings, 1973-01-03 to 2023-06-07
    

    'LTDACBW027SBOG', # Large Time Deposits, All Commercial Banks
    'PMSAVE' # Personal Savings, Jan 1959 to Apr 2023
    
    
         ] 



In [13]:
# series has monthly, weekly and daily frequency variables
# save output in one excel files



# Define start and end dates
start_date = '1986-01-01'
end_date = '2023-03-01'
excel_file_name = 'Data Monthly.xlsx'
sheet_name = 'StessTest'



 # Retrieve the data for each series ID
data = {}
for series_id in series_list:
    data[series_id] = fred.get_series(series_id,observation_start= start_date, observation_end=end_date ,
                 frequency ='m', aggregation_method = 'avg')


# create a new sheet in the Excel file using the pd.ExcelWriter() method, 
# with the vintage date as the sheet name. 

with pd.ExcelWriter(excel_file_name, mode='a', engine='openpyxl') as writer:
    writer.book = load_workbook(excel_file_name)


    # If a sheet with the same name already exists in the Excel file,
   # remove it first before creating the new sheet. 

    if sheet_name in writer.book.sheetnames:
        writer.book.remove(writer.book[sheet_name])
    df = pd.DataFrame(data)
    df.insert(0, 'Date', df.index)

    # save the data to the new sheet 
    df.to_excel(writer, sheet_name=sheet_name, index=False)

## Data for Replication Paper JAE

In [10]:
# Load the original CSV file
file_path = 'FRED_MD.csv'
df_csv = pd.read_csv(file_path)

# monthly data from FRED-MD
series_list = [
    
    ######## Output and Income ##############
    
    'INDPRO', # Industrial Production: Total Index
    'CUMFNS', # Capacity Utilization: Total Index
    'HOUST', # New Privately-Owned Housing Units Started: Total Units
    'W875RX1', # Real Personal Income ex. Current Transfer Receipts
    'DPCERA3M086SBEA', # Real personal consumption expenditures (chain-type quantity index)
    
    ############ Labor Market ###############
    
    'PAYEMS', # All employees, total nonfarm
    'UNRATE', # Unemployment Rate
    'CES0600000008', # Average Hourly Earnings of Production and Nonsupervisory Employees, Goods-Producing
    
    ########## Prices ###################
    
    'CPIAUCSL', # CPI-Urban, All items
    'CPIULFSL', # CPI-Urban: All Items Less Food 
    'PCEPI', # PCE: Chain-type Price Index
    'DSERRG3M086SBEA', # PCE Services: Chain-Type Price Index
    'PPICMM', # PPI by Commodity: Metals and Metal Products
    'OILPRICEx', # Crude Oil, spliced WTI and Cushing 
    
   ######### Interest Rates ##################
    
    'GS10', # 10 Year Treasury Bond Yield
    'FEDFUNDS' # Federal Funds Rate
    'AAA', # Moody's Seasoned Aaa Corporate Bond Yield
    'BAA', # Moody's Seasoned Baa Corporate Bond Yield
    
   
   #########  Stock Market ##############
    
    'S&P 500', # S&P 500 Index  
    'VIXCLSx' # CBOE Volatility Index: VIX
      ] 

# Filter the columns based on the series list and keep the date column
selected_columns = ['sasdate'] + [col for col in series_list if col in df_csv.columns]
df_filtered = df_csv[selected_columns]

# Remove the first row that contains transformation information
df_filtered = df_filtered.drop(0)

# Convert 'sasdate' to a datetime object
df_filtered['sasdate'] = pd.to_datetime(df_filtered['sasdate'])

# Rename the 'sasdate' column to 'Date'
df_filtered.rename(columns={'sasdate': 'Date'}, inplace=True)

# Drop rows with missing values before the date with no missing values for any variable
df_filtered = df_filtered.dropna(subset=selected_columns[1:], how='any').reset_index(drop=True)

# Forward fill (pad) to fill missing values using preceding known values
df_filtered_filled = df_filtered.ffill()

# Backfill (bfill) to fill any remaining missing values using following known values
df_filtered_filled = df_filtered_filled.bfill()


# Python interpreted dates of 21st century (e.g., 2062 instead of 1962). 
# Since this didn't match the expected historical context of the data, 
# the code corrects these future dates by subtracting 100 years, effectively shifting them back to the 20th century.
current_year = pd.Timestamp.today().year
df_filtered_filled['Date'] = \
df_filtered_filled['Date'].apply(lambda x: x - pd.DateOffset(years=100) if x.year > current_year else x)

# Creating the corrected Excel file with the specified sheet name
output_path = 'RepData.xlsx'
with pd.ExcelWriter(output_path, engine='openpyxl') as writer:
    df_filtered_filled.to_excel(writer, sheet_name='Large Data', index=False)



### June 2023 Version of Replication (Draft 1) 

In [13]:
# Retrieve the data for each series ID

# Define start and end dates
start_date = '1978-01-01'
end_date = '2023-05-01'

# Define the excel file and sheet name within the excel file
excel_file_name = 'Data Monthly.xlsx'
sheet_name = 'Rep Data'


data = {}
for series_id in series_list:
    frequency = 'm'
   
    series_data = fred.get_series(series_id, observation_start=start_date,
       observation_end=end_date, frequency=frequency)

    if frequency == 'd':
        # Aggregate daily data to monthly by taking the average of all the daily data for each month
        series_data = series_data.resample('M').mean()
        
    elif frequency == 'w':    
        series_data = series_data.resample('M').mean()
   
    data[series_id] = series_data
    

# create a new sheet in the Excel file using the pd.ExcelWriter() method, 
# with the vintage date as the sheet name. 

with pd.ExcelWriter(excel_file_name, mode='a', engine='openpyxl') as writer:
    writer.book = load_workbook(excel_file_name)

    # Remove the existing sheet if it exists
    if sheet_name in writer.book.sheetnames:
        writer.book.remove(writer.book[sheet_name])
    
    df = pd.DataFrame(data)
    df.insert(0, 'Date', df.index)

    # save the data to the new sheet 
    df.to_excel(writer, sheet_name=sheet_name, index=False)